![rmotr](https://user-images.githubusercontent.com/7065401/52071918-bda15380-2562-11e9-828c-7f95297e4a82.png)
<hr style="margin-bottom: 40px;">

<img src="resources/bybit_logo.png"
    style="width:400px; float: right; margin: 0 40px 40px 40px;"></img>

# Bybit place trades

Application for entering pending orders from csv files on Bybit futures cryptocurrency exchanges.

[Follow this data in a CSV Spreadsheet](data/trades.csv)

![purple-divider](resources/purple_divider.png)

## Hands on! 

In [1]:
import pandas as pd
import requests
from pybit import usdt_perpetual
from utils import load_config, create_account_info, transform_trades, place_trades_on_exchange, write_trades_to_audit_log

config = load_config("../../../config.yaml")

exchange = usdt_perpetual.HTTP(
    endpoint = config["bybitApi"]["endpoint"],
    api_key = config["bybitApi"]["apiKey"],
    api_secret = config["bybitApi"]["secretKey"]
)

![green-divider](resources/green_divider.png)

## Loading data

In [2]:
trades_raw = pd.read_csv("data/trades.csv")

wallet_balance_raw = exchange.get_wallet_balance(coin = "USDT")

instruments_info = requests.get(config["bybitApi"]["instrumentInfoEndpoint"]).json()

![green-divider](resources/green_divider.png)


## The data at a glance

In [3]:
trades_raw

,ticker,direction,entry_price,stop_loss
0,BTCUSDT,LONG,20000,19800


In [4]:
str(wallet_balance_raw)[:500]

"{'ret_code': 0, 'ret_msg': 'OK', 'result': {'USDT': {'equity': 195.49150376, 'available_balance': 195.49150376, 'used_margin': 0, 'order_margin': 0, 'position_margin': 0, 'occ_closing_fee': 0, 'occ_funding_fee': 0, 'wallet_balance': 195.49150376, 'realised_pnl': 0, 'unrealised_pnl': 0, 'cum_realised_pnl': -12.92849624, 'given_cash': 0, 'service_cash': 0}}, 'ext_code': '', 'ext_info': '', 'time_now': '1674899140.628961', 'rate_limit_status': 119, 'rate_limit': 120, 'rate_limit_reset_ms': 16748991"

In [5]:
str(instruments_info)[:500]

"{'retCode': 0, 'retMsg': 'OK', 'result': {'category': 'linear', 'list': [{'symbol': '10000NFTUSDT', 'contractType': 'LinearPerpetual', 'status': 'Trading', 'baseCoin': '10000NFT', 'quoteCoin': 'USDT', 'launchTime': '1643007175000', 'deliveryTime': '0', 'deliveryFeeRate': '', 'priceScale': '6', 'leverageFilter': {'minLeverage': '1', 'maxLeverage': '12.50', 'leverageStep': '0.01'}, 'priceFilter': {'minPrice': '0.000005', 'maxPrice': '9.999990', 'tickSize': '0.000005'}, 'lotSizeFilter': {'maxTradin"

![green-divider](resources/green_divider.png)


## Transform data

In [6]:
account_info = create_account_info(wallet_balance_raw, config)

account_info

{'balanceOnExchange': 195.49, 'balanceInBank': 1977, 'accountSize': 2172.49}

In [7]:
trades = transform_trades(trades_raw, instruments_info, config, account_info)

trades

,ticker,direction,entry_price,stop_loss,move,profit_target,position,position_usd,leverage,price_scale,expected_profit,expected_loss
0,BTCUSDT,LONG,20000,19800,200,20200,0.10862,2172.49,12.347833,2,21.724,-21.724


![green-divider](resources/green_divider.png)


## Process data

In [8]:
write_trades_to_audit_log(trades, config["bybitPlaceTrades"]["auditLogPath"])

place_trades_on_exchange(trades, exchange)



,trade_id,status
0,BTCUSDT_LONG,OK
